In [1]:
import pandas as pd
import os
import datetime 

pd.set_option('display.max_columns', None)

In [2]:
dataset_names = ['BattersBox','SPRoundup']

for i in dataset_names:
    dirName = i
    try:
        # Create Directory
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ") 
    except FileExistsError:
        print("Directory " , dirName ,  " already exists")

Directory  BattersBox  already exists
Directory  SPRoundup  already exists


In [3]:
year = 2020
month = 8
day = 3
previous_num_days = 4

base = datetime.date(year, month, day)
date_list = [base - datetime.timedelta(days=x) for x in range(previous_num_days)]

In [4]:
for i in date_list:
    
    !python BattersBox.py --date $i
    
    !python StartingPitcherRoundup.py --date $i

Generating Batters Box data for 2020-08-03
Generating SP Roundup data for 2020-08-03
Generating Batters Box data for 2020-08-02
Generating SP Roundup data for 2020-08-02
Generating Batters Box data for 2020-08-01
Generating SP Roundup data for 2020-08-01
Generating Batters Box data for 2020-07-31
Generating SP Roundup data for 2020-07-31


In [5]:
files = []
for i in dataset_names:
    
    file_name = i
    for j in date_list:
        try:
            
            file = f'{file_name}{str(j)}.xls'
            path = f'./{file_name}/{file_name}{str(j)}.xls'
            os.rename(file, path)
            files.append(path)
            print(file, 'Moved')
        except:
            print(file , 'File does not exist')

BattersBox2020-08-03.xls Moved
BattersBox2020-08-02.xls Moved
BattersBox2020-08-01.xls Moved
BattersBox2020-07-31.xls Moved
SPRoundup2020-08-03.xls Moved
SPRoundup2020-08-02.xls Moved
SPRoundup2020-08-01.xls Moved
SPRoundup2020-07-31.xls Moved


In [7]:
# Batters

li = []

for filename in files:
    if filename[2] == 'B':
        batters = pd.read_excel(filename,skiprows=[0])
        batters = batters.loc[:,:'SB']
        batters['date'] = filename[23:33]

        li.append(batters)

batters = pd.concat(li, axis=0, ignore_index=True)

In [8]:
# Pitchers

li = []

for filename in files:
    if filename[2] == 'S':
        pitchers = pd.read_excel(filename,skiprows=[0])
        pitchers = pitchers.loc[:,'Player':'CSW%.7']
        pitchers['date'] = filename[21:31]
        
        li.append(pitchers)

pitchers = pd.concat(li, axis=0, ignore_index=True)

In [10]:
def draft_king_batters(single, double, triple, HR, RBI, R, BB, HBP, SB):
    
    return (3 * single) + (5 * double) + (8 * triple) + (10 * HR) + (2 * RBI) + (2 * R) + (2 * BB) + (2 * HBP) + (5 * SB)



batters['draft_kings_points'] = batters.apply(lambda batter: draft_king_batters(batter['1B'],
                                                                                batter['2B'],
                                                                                batter['3B'],
                                                                                batter['HR'],
                                                                                batter['RBI'],
                                                                                batter['R'],
                                                                                (batter['BB'] + batter['IBB']),
                                                                                batter['HBP'],
                                                                                batter['SB']),axis=1)

In [14]:
def draft_king_pitchers(IP,K,W,ER,H,BB,HBP):
    
    tot = (2.25 * IP) + (2 * K) + (-2 * ER) + (-0.6 * H) + (-0.6 * BB) + (-0.6 * HBP)
    
    # Win
    if W == 'W':
        tot +=4
    
    # CG
    if IP == 9:
        tot += 2.5
        
    # CGS
    if IP == 9 and ER == 0:
        tot +=2.5
        
    # No Hitter
    if IP == 9 and H == 0:
        tot += 5
    return tot




pitchers['draft_kings_points'] = pitchers.apply(lambda pitcher: draft_king_pitchers(pitcher['IP'],
                                                                                pitcher['K'],
                                                                                pitcher['Dec'],
                                                                                pitcher['ER'],
                                                                                pitcher['H'],
                                                                                pitcher['BB'],
                                                                                0),axis=1)

In [18]:
batters.head()

,Player,Statline,PA,AB,H,1B,2B,3B,HR,BB,IBB,HBP,K,R,RBI,SB,date,draft_kings_points
0,Wilson Ramos,"2-4, HR, R, 3 RBI",4,4,2,1,0,0,1,0,0,0,0,1,3,0,2020-08-03,21
1,Khris Davis,"2-3, HR, 2 R, 2 RBI, BB",5,3,2,1,0,0,1,1,0,1,0,2,2,0,2020-08-03,25
2,Michael Conforto,"3-5, 2B, 2 R, RBI, 2 SB",5,5,3,2,1,0,0,0,0,0,0,2,1,2,2020-08-03,27
3,Yoan Moncada,"3-8, HR, 2B, 2 R, RBI",8,8,3,1,1,0,1,0,0,0,2,2,1,0,2020-08-03,24
4,Kris Bryant,"2-3, HR, 2B, R, RBI, BB",4,3,2,0,1,0,1,1,0,0,0,1,1,0,2020-08-03,21


In [19]:
batters.groupby('Player').mean().head()

,PA,AB,H,1B,2B,3B,HR,BB,IBB,HBP,K,R,RBI,SB,draft_kings_points
Player,,,,,,,,,,,,,,,
AJ Pollock,3.75,3.500000,0.500000,0.25,0.000000,0.0,0.25,0.250000,0.0,0.0,1.500000,0.250000,0.500000,0.0,5.250000
Aaron Hicks,4.00,2.666667,0.333333,0.00,0.333333,0.0,0.00,1.333333,0.0,0.0,0.666667,0.333333,0.333333,0.0,5.666667
Aaron Judge,5.00,4.500000,1.500000,0.25,0.250000,0.0,1.00,0.500000,0.0,0.0,1.000000,1.250000,2.000000,0.0,19.500000
Abraham Almonte,4.00,3.000000,1.000000,1.00,0.000000,0.0,0.00,1.000000,0.0,0.0,1.000000,0.000000,0.000000,2.0,15.000000
Abraham Toro,3.00,3.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.000000


In [17]:
pitchers.head()

,Player,IP,ER,H,K,BB,Dec,#,CSW,CSW%,#.1,MPH,CSW%.1,#.2,MPH.1,CSW%.2,#.3,MPH.2,CSW%.3,#.4,MPH.3,CSW%.4,#.5,MPH.4,CSW%.5,#.6,MPH.5,CSW%.6,#.7,MPH.6,CSW%.7,date,draft_kings_points
0,Alec Mills,7.0,0,3,4,3,W,98,22,22.4,48,90.3,20.8,18,80.2,27.8,11,66.0,18.2,21,78.2,23.8,0,0.0,0.0,0,0.0,0.0,0,0,0,2020-08-03,24.15
1,Frankie Montas,7.0,1,4,9,4,W,99,44,44.4,55,95.1,40.0,0,0.0,0.0,0,0.0,0.0,26,85.9,53.8,0,0.0,0.0,18,85.9,44.4,0,0,0,2020-08-03,30.95
2,Sonny Gray,6.0,1,4,8,2,W,102,34,33.3,51,92.8,31.4,5,90.0,40.0,34,81.2,29.4,12,83.1,50.0,0,0.0,0.0,0,0.0,0.0,0,0,0,2020-08-03,27.90
3,Danny Duffy,6.0,1,3,6,4,L,103,32,31.1,56,91.9,33.9,19,84.7,42.1,11,76.3,0.0,17,83.5,29.4,0,0.0,0.0,0,0.0,0.0,0,0,0,2020-08-03,19.30
4,Gerrit Cole,6.0,1,5,4,1,W,91,23,25.3,46,96.9,19.6,3,87.7,0.0,18,83.6,44.4,24,87.8,25.0,0,0.0,0.0,0,0.0,0.0,0,0,0,2020-08-03,19.90
